In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
pd.options.display.float_format = '{:.2f}'.format

from itertools import combinations
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from prophet import Prophet

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Measles/cameroon.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Year-Month  159 non-null    object
 1   Cases       159 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.6+ KB


In [5]:
df

,Year-Month,Cases
0,2011-1,58
1,2011-2,147
2,2011-3,186
3,2011-4,105
4,2011-5,114
...,...,...
154,2023-11,16
155,2023-12,25
156,2024-1,73
157,2024-2,113


In [6]:
df.columns = ['ds','y']

In [7]:
train_data_pr = df.iloc[:len(df)-12]
test_data_pr = df.iloc[len(df)-12:]

In [8]:
len(test_data_pr)

12

In [9]:
test_data_pr

,ds,y
147,2023-4,1205
148,2023-5,917
149,2023-6,448
150,2023-7,98
151,2023-8,16
152,2023-9,16
153,2023-10,31
154,2023-11,16
155,2023-12,25
156,2024-1,73


In [10]:
from enum import auto
#m = Prophet()
m = Prophet(seasonality_mode = "multiplicative",growth='flat',yearly_seasonality=False,weekly_seasonality=False,daily_seasonality=False,
            changepoints=None, n_changepoints=5, changepoint_range=0)
m.fit(train_data_pr)
future = m.make_future_dataframe(periods=12,freq='MS')
prophet_pred = m.predict(future)

INFO:prophet:n_changepoints greater than number of observations. Using -1.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpy58hwp15/ti7j0sru.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpy58hwp15/07wbf8k3.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=77074', 'data', 'file=/tmp/tmpy58hwp15/ti7j0sru.json', 'init=/tmp/tmpy58hwp15/07wbf8k3.json', 'output', 'file=/tmp/tmpy58hwp15/prophet_model11ppphsq/prophet_model-20240720070833.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:08:33 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:08:33 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [11]:
prophet_pred.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
154,2023-11-01,127.57,-103.64,374.29,127.57,127.57,0.00,0.00,0.00,0.00,0.00,0.00,127.57
155,2023-12-01,127.57,-108.31,395.07,127.57,127.57,0.00,0.00,0.00,0.00,0.00,0.00,127.57
156,2024-01-01,127.57,-134.50,378.78,127.57,127.57,0.00,0.00,0.00,0.00,0.00,0.00,127.57
157,2024-02-01,127.57,-127.22,357.34,127.57,127.57,0.00,0.00,0.00,0.00,0.00,0.00,127.57
158,2024-03-01,127.57,-129.60,394.79,127.57,127.57,0.00,0.00,0.00,0.00,0.00,0.00,127.57


In [12]:
prophet_pred = pd.DataFrame({"Date" : prophet_pred[-12:]['ds'], "Pred" : prophet_pred[-12:]["yhat"]})

In [13]:
prophet_pred = prophet_pred.set_index("Date")

In [14]:
prophet_pred.index.freq = "MS"

In [15]:
prophet_pred

,Pred
Date,
2023-04-01,127.57
2023-05-01,127.57
2023-06-01,127.57
2023-07-01,127.57
2023-08-01,127.57
2023-09-01,127.57
2023-10-01,127.57
2023-11-01,127.57
2023-12-01,127.57


In [16]:
from sklearn.metrics import mean_squared_error
error = np.sqrt(mean_squared_error(test_data_pr['y'],prophet_pred['Pred']))
#print('Test RMSE: %.4f' % error)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(test_data_pr['y'],prophet_pred['Pred'])
#print("Mean Absolute Error (MAE):", mae)

from sklearn.metrics import mean_squared_log_error
msle = mean_squared_log_error(test_data_pr['y'],prophet_pred['Pred'])
#print("Mean Squared Logarithmic Error (MSLE):", msle)

from sklearn.metrics import mean_absolute_percentage_error
#print('Mean Absolute Percentage Error (MAPE):',mean_absolute_percentage_error(valid["Cases"],predictions))

print('RMSE: %.2f' % error)
def mean_absolute_percentage_error(y_true, y_pred):

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


mape = mean_absolute_percentage_error(test_data_pr['y'], prophet_pred['Pred'])
print(f"MAPE: {mape:.2f}%")
print('MAE:  %.2f' % mae)
print('MSLE: %.2f' % msle)



RMSE: 404.36
MAPE: 374.67%
MAE:  244.87
MSLE: 2.84
